In [10]:
import numpy as np
import xlrd
import xlwt
import math

In [55]:
spot = xlrd.open_workbook('Zinc_spot.xlsx')
futures = xlrd.open_workbook('ZInc_3_futures.xlsx')

In [56]:
tables = spot.sheets()[0]
tablef = futures.sheets()[0]

In [57]:
num_of_rows_spot = tables.nrows
num_of_rows_futures = tablef.nrows

In [58]:
y = 2019
m  = 4
i = 0
j = 1
spot_price_tmp = []
spot_vol = {}
log_return_tmp = []
while i < num_of_rows_spot:
    tup = xlrd.xldate_as_tuple(tables.cell(i,0).value, 0)
    yi = tup[0]
    mi = tup[1]
    #di = tup[2]
    #print(yi, mi, di)
    if (yi == y and mi == m):
        spot_price_tmp.append(tables.cell(i,1).value)
        i = i+1
    else:
        tmp = str(y)+"/"+str(m)
        #print(spot_price_tmp)
        while j < len(spot_price_tmp):
            log_return_tmp.append(math.log(spot_price_tmp[j-1]/spot_price_tmp[j]))
            j = j+1
        #print(log_return_tmp)
        spot_vol[tmp] = np.std(log_return_tmp)*math.sqrt(252)
        spot_price_tmp.clear()
        log_return_tmp.clear()
        m = m-1
        j = 1
        if m == 0:
            y = y -1
            m = 12

y = 2019
m = 4
spot_price_tmp = []
log_return_tmp = []
i=0           
futures_vol = {}
while i < num_of_rows_futures:
    tup = xlrd.xldate_as_tuple(tablef.cell(i,0).value, 0)
    yi = tup[0]
    mi = tup[1]
    #di = tup[2]
    #print(yi, mi, di)
    if (yi == y and mi == m):
        spot_price_tmp.append(tablef.cell(i,1).value)
        i = i+1
    else:
        tmp = str(y)+"/"+str(m)
        #print(spot_price_tmp)
        while j < len(spot_price_tmp):
            log_return_tmp.append(math.log(spot_price_tmp[j-1]/spot_price_tmp[j]))
            j = j+1
        #print(tmp)
        #print(log_return_tmp)
        futures_vol[tmp] = np.std(log_return_tmp)*math.sqrt(252)
        spot_price_tmp.clear()
        log_return_tmp.clear()
        m = m-1
        j = 1
        if m == 0:
            y = y -1
            m = 12

excess_vol = {}
for d in spot_vol:
    excess_vol[d] = (spot_vol[d]-futures_vol[d])/futures_vol[d]

In [59]:
book = xlwt.Workbook()
sheet = book.add_sheet('volatility')
row = 1
sheet.write(0,1,"spot")
sheet.write(0,2,"futures")
sheet.write(0,3,"excess")
for d in spot_vol:
    sheet.write(row,0,d)
    sheet.write(row,1,spot_vol[d])
    sheet.write(row,2,futures_vol[d])
    sheet.write(row,3,excess_vol[d])
    row = row +1

book.save('Zinc volatility.xls')